In [1]:
import os
import random
from datetime import datetime

import pandas as pd

## Task 1
Cho trước 1 folder gồm các file ở định dạng .csv và .xlsx, hãy thực hiện merge tất cả các file .csv thành 1 file duy nhất

<b>Kiến thức:</b> trong python có module <b><i>os</i></b> giúp các bạn làm việc với hệ điều hành như tạo folder, lấy danh sách file, ...
- ```os.path.exists(path)```: kiểm tra directory truyền vào có tồn tại hay không, nếu đã có return True và ngược lại
- ```os.makedirs(path)```: tạo folder theo đường dẫn truyền vào, nếu đã có sẽ báo lỗi do đó nên đi kèm với điều kiện kiểm tra tồn tại hay chưa
- ```os.listdir(path)```: liệt kê danh sách các file, subfolder trong đường dẫn (chỉ ở mức level 1 nếu trong subfolder còn file sẽ không liệt kê được)
- ```os.walk(path)```: tương tự như os.listdir() nhưng liệt kê được đầy đủ tất cả file trong subfolder

#### Đoạn code sau tạo dữ liệu giả để sử dụng cho task 1 (chỉ run 1 lần duy nhất), các file được tạo sẽ nằm trong folder fake_data trong cùng thư mục chứa file notebook này

In [79]:
colors = ['red', 'green', 'yellow', 'blue', 'cyan', 'pink', 'black', 'white',]

if not os.path.exists('fake_data/'):
    os.makedirs('fake_data/')

for i in range(5):
    random_colors = random.choices(colors, k = 5)
    df = pd.DataFrame(random_colors, columns=['colors'])
    df.to_csv(f"fake_data/{datetime.now().strftime('%d_%m_%Y')}_{i}.csv", index=False)
    df.to_excel(f"fake_data/{datetime.now().strftime('%d_%m_%Y')}_{i}.xlsx", index=False)

In [4]:
# Start here
# folder_path = "D:\Study\JupyterNotebook" # Đây là folder chính anh lưu bài tập
folder_path = os.getcwd()  # Giống dòng trên nhưng không cần khai báo cụ thể path (cwd: current working directory) 

dataframes = []  # Tạo 1 danh sách để lưu các DataFrame

for file in os.listdir(folder_path): 
    if file[-4:] == '.csv' and file != 'merged_data.csv' :  # Lọc file CSV
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)  # Đọc file CSV
        dataframes.append(df)  # Thêm vào danh sách

merged_df = pd.concat(dataframes)
# merged_df = pd.concat(
#     (pd.read_csv(os.path.join(folder_path, file)) 
#     for file in os.listdir(folder_path) 
#     if file[-4:] == '.csv' and file != 'merged_data.csv')  # Để reset lại chỉ số của DataFrame gộp
# )

output_path = os.path.join(folder_path, 'merged_data.csv')
merged_df.to_csv(output_path, index=False) # Lưu DataFrame gộp thành file CSV
print(f"Đã gộp file CSV thành công và lưu tại: {output_path}")

Đã gộp file CSV thành công và lưu tại: d:\Study\JupyterNotebook\Homework_7\merged_data.csv


## Task 2
Tương tự như task 1 tuy nhiên yêu cầu là tạo 1 file excel kết quả DUY NHẤT gồm nhiều sheet trong đó mỗi sheet là nội dung của mỗi file csv (tên sheet sẽ là tên file csv).

<b>Kiến thức</b>: Để tạo file excel ta có thể sử dụng ```df.to_excel()``` tuy nhiên cách này không thể tạo nhiều sheet trong 1 file được.<br>
Để tạo nhiều sheet ta thực hiện như sau:
```
>>> writer = pd.ExcelWriter('đường dẫn kèm tên file kết quả')  # tạo writer
>>> df1.to_excel(writer, sheet_name='Sheet1')  # sử dụng df1 để tạo sheet1 trong writer
>>> df2.to_excel(writer, sheet_name='Sheet2')  # sử dụng df2 để tạo sheet2 trong writer
>>> writer.save()  # ghi file
```

Bạn có thể sử dụng hàm for với cách này để tạo nhiều sheet trong vòng for.

In [84]:
# Start here
folder_path = os.getcwd()  
output_path = os.path.join(folder_path, "merged_data.xlsx")

writer = pd.ExcelWriter(output_path)

for file in os.listdir(folder_path):
    if file[-4:] == '.csv' and file != 'merged_data.csv':
        file_path = os.path.join(folder_path, file)  
        df = pd.read_csv(file_path)  # Đọc file CSV
        df.to_excel(writer, sheet_name=f"Sheet{len(writer.sheets) + 1}", index=False)  # Tạo sheet

# Ghi file Excel
writer.close()  # https://pandas.pydata.org/docs/reference/api/pandas.ExcelWriter.html
print(f"Đã tạo file Excel thành công và lưu tại: {output_path}")

Đã tạo file Excel thành công và lưu tại: d:\Study\JupyterNotebook\merged_data.xlsx


## Task 3
Cho 2 file csv trong đó:
- transaction.csv là file chứa thông tin các giao dịch của customer
- fraud_customer.csv là file chứa id của các customer được coi là fraud

Hãy lấy ra các giao dịch của các customer không nằm trong danh sách fraud và chia làm 2 sheet với:
- sheet tên big chứa các giao dịch lớn hơn bằng 9000000
- sheet tên small chứa các giao dịch nhỏ hơn 9000000

Sau đó bổ sung thêm 1 sheet thứ 3 với tên là fraud_tx chứa các giao dịch của customer nằm trong danh sách fraud và được sắp xếp theo cột 'amount' giảm dần. 

<b>Lưu ý: Các sheet cần được lưu trong cùng 1 file excel gồm big, small & fraud_tx.</b>

In [99]:
# Start here
folder_path = os.getcwd() 
transactions_file = os.path.join(folder_path,"transactions.csv")
fraud_file = os.path.join(folder_path,"fraud_customer.csv")

# Đọc dữ liệu từ các file CSV
transactions_df = pd.read_csv(transactions_file)
fraud_customers_df = pd.read_csv(fraud_file)

# Lọc ra các giao dịch của customer không nằm trong danh sách fraud
non_fraud_transactions = transactions_df[~transactions_df['customer_id'].isin(fraud_customers_df['customer_id_fraud'])]

# Tạo DataFrame cho các giao dịch lớn và nhỏ
big_transactions = non_fraud_transactions[non_fraud_transactions['amount'] >= 9000000]
small_transactions = non_fraud_transactions[non_fraud_transactions['amount'] < 9000000]

# Tạo DataFrame cho các giao dịch của customer fraud và sắp xếp theo amount giảm dần
fraud_transactions = transactions_df[transactions_df['customer_id'].isin(fraud_customers_df['customer_id_fraud'])].sort_values('amount', ascending=False)

# Đường dẫn lưu file Excel
output_file = os.path.join(folder_path,"filtered_transactions.xlsx")

# Lưu vào file Excel với nhiều sheet
with pd.ExcelWriter(output_file) as writer:
    big_transactions.to_excel(writer, sheet_name='big', index=False)
    small_transactions.to_excel(writer, sheet_name='small', index=False)
    fraud_transactions.to_excel(writer, sheet_name='fraud_tx', index=False)

print("Đã xuất file Excel thành công!")

Đã xuất file Excel thành công!
